In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize, WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, GlobalMaxPool1D, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping



In [2]:
#tylko za 1 razem
#nltk.download('wordnet')
#nltk.download('punkt')

In [3]:
# Wczytywanie danych
train_data = pd.read_csv("jigsaw-toxic-comment-classification-challenge/train.csv")
train_data["comment_text"] = train_data["comment_text"].str.lower()
#print(train_data["comment_text"])
print("pre clean:\n\n", train_data["comment_text"])

pre clean:

 0         explanation\nwhy the edits made under my usern...
1         d'aww! he matches this background colour i'm s...
2         hey man, i'm really not trying to edit war. it...
3         "\nmore\ni can't make any real suggestions on ...
4         you, sir, are my hero. any chance you remember...
                                ...                        
159566    ":::::and for the second time of asking, when ...
159567    you should be ashamed of yourself \n\nthat is ...
159568    spitzer \n\numm, theres no actual article for ...
159569    and it looks like it was actually you who put ...
159570    "\nand ... i really don't think you understand...
Name: comment_text, Length: 159571, dtype: object


In [4]:
# Preprocessing danych
def cleaning(data):
    # remove the characters in the first parameter
    clean_column = re.sub('<.*?>', ' ', str(data))
    # removes non-alphanumeric characters except periods.
    clean_column = re.sub('[^a-zA-Z0-9.]+', ' ', clean_column)
    # tokenize
    tokenized_column = word_tokenize(clean_column)
    return tokenized_column

In [5]:
train_data["cleaned"] = train_data["comment_text"].apply(cleaning)
print("post clean:\n\n", train_data["cleaned"])

post clean:

 0         [explanation, why, the, edits, made, under, my...
1         [d, aww, he, matches, this, background, colour...
2         [hey, man, i, m, really, not, trying, to, edit...
3         [more, i, can, t, make, any, real, suggestions...
4         [you, sir, are, my, hero, ., any, chance, you,...
                                ...                        
159566    [and, for, the, second, time, of, asking, when...
159567    [you, should, be, ashamed, of, yourself, that,...
159568    [spitzer, umm, theres, no, actual, article, fo...
159569    [and, it, looks, like, it, was, actually, you,...
159570    [and, ..., i, really, don, t, think, you, unde...
Name: cleaned, Length: 159571, dtype: object


In [ ]:
# Lematyzacja slow
lemmatizer = WordNetLemmatizer()

def lemmatizing(data):
    # input our data in function, take the cleaned column
    my_data = data["cleaned"]
    lemmatized_list = [lemmatizer.lemmatize(word) for word in my_data]
    return (lemmatized_list)

train_data["lemmatized"] = train_data.apply(lemmatizing, axis=1)
print("post lemmatize:\n\n", train_data["lemmatized"])

train_data["comment_text"] = train_data["lemmatized"]
train = train_data[["comment_text"]]
train_labels = train_data[["toxic"]]
print(train)

In [ ]:
 # 2. Use train_test_split to split into train/test
comment_train, comment_test, labels_train, labels_test = train_test_split(train, train_labels, test_size=0.2,
                                                                          random_state=42)
# Transpose and flatten so it fits the correct dimensions
labels_train = np.transpose(labels_train)
labels_train = np.ravel(labels_train)
labels_test = np.transpose(labels_test)
labels_test = np.ravel(labels_test)

In [ ]:
# 3. CountVectorizer
count_vect = CountVectorizer()
comment_train_counts = count_vect.fit_transform(comment_train.comment_text.astype(str))
print(comment_train_counts)

In [ ]:
#4. TfidfTransformer
#Use tf-idf instead
tf_transformer = TfidfTransformer(use_idf=False).fit(comment_train_counts)
comment_train_tf = tf_transformer.transform(comment_train_counts)

tfidf_transformer = TfidfTransformer()
comment_train_tfidf = tfidf_transformer.fit_transform(comment_train_counts)

In [ ]:
# Naive Bayes classifier
# tworzenie modelu
clf = MultinomialNB().fit(comment_train_tfidf, labels_train)

# pakowanie slow w paczki
comment_test_new_counts = count_vect.transform(comment_test.comment_text.astype(str))
comment_test_new_tfidf = tfidf_transformer.transform(comment_test_new_counts)

In [ ]:
#6 Trenowanie modelu LSTM
num_words = 20000
max_len = 150
emb_size = 128

# vectorize a text corpus, by turning each text into either a sequence of integers (each integer being the index of a token in a dictionary) 
# or into a vector where the coefficient for each token could be binary, based on word count, based on tf-idf
tok = Tokenizer(num_words = num_words, lower=True, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tok.fit_on_texts(list(comment_train.comment_text.astype(str)))

comment_train2 = tok.texts_to_sequences(comment_train.comment_text.astype(str))
comment_test2 = tok.texts_to_sequences(comment_test.comment_text.astype(str))

comment_train2 = sequence.pad_sequences(comment_train2, maxlen = max_len)
comment_test2 = sequence.pad_sequences(comment_test2, maxlen = max_len)


inp = Input(shape = (max_len, ))
layer = Embedding(num_words, emb_size)(inp)
layer = Bidirectional(LSTM(50, return_sequences = True, recurrent_dropout = 0.15))(layer)
layer = GlobalMaxPool1D()(layer)
layer = Dropout(0.2)(layer)
layer = Dense(50, activation = 'relu')(layer)
layer = Dropout(0.2)(layer)
layer = Dense(1, activation = 'sigmoid')(layer)
model = Model(inputs = inp, outputs = layer)
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
print("\n")
model.summary()

file_path = 'save_best_model.keras'
checkpoint = ModelCheckpoint(file_path, monitor = 'val_loss', verbose = 1, save_best_only=True)
early_stop = EarlyStopping(monitor = 'val_loss', patience = 1)

model.fit(comment_train2, labels_train, batch_size = 512, epochs = 1, validation_split = 0.2, validation_data = (comment_test2, labels_test), callbacks = [checkpoint, early_stop])
